In [71]:
#-------------------------------------IMPORTS-------------------------------------#
import torch
import gzip
import numpy as np
from random import randint
import torch.utils.data as data_utils
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics.pairwise import cosine_similarity
from numpy import linalg as LA

In [144]:
#-------------------------------------GLOBAL VARIABLES-------------------------------------#
NUM_TRAINING_EXAMPLES = 1000 #FOR DATA BATCHER, WHEN DEPLOYED SHOULD BE ALL TRAINING EXAMPLES
PARAMETER_MAX_TITLE_LENGTH = 38
NUM_FEATURES_PER_WORD = 200 #DO NOT CHANGE. FIXED at 200

In [195]:
#-------------------------------------HELPER FUNCTIONS-------------------------------------#
NUM_NEGATIVE_SAMPLES=20

def N_random_values_in_list(full_list, N):
    x=0
    lower_bound  = 0
    upper_bound = len(full_list)-1
    sample_list=[]
    random_nums=[]
    while x < min(N,len(full_list)):
        random_num = randint(lower_bound, upper_bound) # inclusive range
        if random_num in random_nums:
            continue
        else:
            random_nums.append(random_num)
            x += 1
    return [full_list[i] for i in random_nums]

def convert_to_list(filename):
    if filename.endswith('gz'):
        with gzip.open(filename,'r')as f:
            text_tokens = f.readlines()
    else:
        with open(filename, 'r') as f:
            text_tokens = f.readlines()
    text_tokens = [token.replace('\n','').split('\t') for token in text_tokens]
    text_tokens = [[token[0], token[1].split(' '), token[2].split(' ')] for token in text_tokens]
                   
    return text_tokens

#Sample:question_id, similar_question_id, negative_question_id
def convert_to_samples(filename):
    my_list=convert_to_list(filename)
    new_samples=[]
    for original_sample in my_list:
        for similar in original_sample[1]:
            random_negative_samples = N_random_values_in_list(original_sample[2],NUM_NEGATIVE_SAMPLES)
            new_samples.append([original_sample[0], similar, random_negative_samples])# change this to include all negative 
                                                                                     # examples later
    return new_samples
def make_lookup_table_for_training_data(filename):
    lookup={}
    text_token_list=convert_to_list(filename)
    for token in text_token_list:
        lookup[token[0]] = {'title':token[1],'question':token[2]}
    return lookup
        
#takes  sample_ids of [[q1,p1,n1],[q2,p2,n2]....]
#outputs titles like [[q1_title, p1_title, n1_title],[q2_title,p2_title,n2_title]...]
def convert_sampleids_to_titles(sample_ids,lookup):
    #each sample_id [question_id, pos_id, [neg_ids]]
    #print type(sample_ids)==list, "first"
   
    titles = []
    for sample_id in sample_ids:
        
         #flatten list: [question_id, pos_id, [neg_ids]] --> [question_id, pos_id, neg_id1, neg_id2, ...]
        sample_id= sample_id[:2]+sample_id[2][:]
        #sample_id : question_id, similar_question_id, negative_question_id
        try:
            titles.append([lookup[str(identity)]['title'] for identity in sample_id])
           # print type(sample_id)==list
        except:
            print sample_id, "is sample id", type(sample_id)==list
    return titles
def remove_non_ascii(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

def extract_features(word):
    try:
        word=remove_non_ascii(word)
        word=word.encode('utf-8')
    except:
        print(word)
    return word_to_vec.get(word,[0.0 for i in range(200)])

def find_maximum_title_and_body_length(lookup_table):
    max_len_title = -1
    max_len_question = -1
    max_len_question_id = 0
    for key, dict_val in lookup_table.iteritems():
        len_title = len(dict_val['title'])
        len_question = len(dict_val['question'])
        if len_title > max_len_title:
             max_len_title = len_title
        if len_question > max_len_question:
            max_len_question = len_question
            max_len_question_id = key
    return max_len_title, max_len_question

def title_to_feature_matrix(title_word_list):
    feature_matrix = []
    for idx, word in enumerate(title_word_list):
        if idx == PARAMETER_MAX_TITLE_LENGTH:
            break
        else:
            word_features = extract_features(word)
            feature_matrix.append(word_features)
        
    #Pad the feature with zeros to ensure all inputs to the net have the same dimension
    feature_matrix += [[0] * NUM_FEATURES_PER_WORD] * (PARAMETER_MAX_TITLE_LENGTH - len(title_word_list))
    #print np.array(feature_matrix).T.shape
    return np.array(feature_matrix).T



#array is structured like a batch of features 50x200x38
def find_start_of_padding_for_batch(batch):
    vec_lengths_in_batch = []
    for batch_num in range(0, len(batch)):
        single_vec = batch[batch_num]
        length = find_start_of_padding_single_vec(single_vec) + 1
        vec_lengths_in_batch.append(length)
    return vec_lengths_in_batch

#batch = 200x38
def find_start_of_padding_single_vec(single_vec):
    for idx in range(len(single_vec[0])-1, -1, -1):
        if single_vec[0][idx] != 0.:
            return idx
    #if the whole sequence is 0s
    return 0
def create_mask(word_length):
    return np.array([[1. / word_length] * HIDDEN_SIZE] * word_length + [[0] * HIDDEN_SIZE] * (MAX_TITLE_LENGTH - word_length)).T

In [81]:
#-------------------------------------LOAD DATA-------------------------------------#

word_embeddings = 'askubuntu/vector/vectors_pruned.200.txt.gz'
f = gzip.open(word_embeddings, 'r')
wv_text = [ ]
lines = f.readlines()
for line in lines:
    wv_text.append(line.strip())

word_to_vec = {}

for line in wv_text:
    parts = line.split()
    word = parts[0]
    vector = np.array([float(v) for v in parts[1:]])
    word_to_vec[word] = vector
f.close()


#text_tokenized.txt.gz has id \t title \t question body
text_tokenized='askubuntu/text_tokenized.txt.gz'

#train_random.txt
#(1) the query question ID, (2) the list of similar question IDs, and (3) the list of randomly selected question IDs.
train_random_filename='askubuntu/train_random.txt'

#Each line contains (1) the query question ID, (2) the list of similar question IDs, (3) the list of 20 candidate question IDs and (4) the associated BM25 scores of these questions computed by the Lucene search engine. The second field (the set of similar questions) is a subset of the third field.
dev_filename='askubuntu/dev.txt'
test_filename='askubuntu/test.txt'

train_samples = convert_to_samples(train_random_filename)
dev_samples = convert_to_samples(dev_filename)
test_samples = convert_to_samples(test_filename)

lookup = make_lookup_table_for_training_data(text_tokenized)
train_titles_only = convert_sampleids_to_titles(train_samples, lookup)

In [154]:
#-------------------------------------CREATE DATA BATCHER-------------------------------------#
#for each tuple of titles make a feature vector that is num_titles x 200 x 38
# where num_titles = 1 (target) + 1 (positive) + n (negative) 
features = []
for i in range(len(train_titles_only[:NUM_TRAINING_EXAMPLES])):   # we should include all but this is just for simplicity 
    if i%1000 == 0:
        print i

    sample = train_titles_only[i]
    
    target_title = sample[0]
    positive_title = sample[1]
    negative_titles = sample[2:]
    
#     print "\n"
#     print "Target title: {}".format(" ".join(target_title))
#     print "Positive title: {}".format(" ".join(positive_title))
#     for negative in negative_titles:
#         print "Negative title: {}".format(" ".join(negative))
#     print "\n"
#     a
#     if i > 10:
#         a
#     print "target_title: {}".format(target_title)
#     print "positive title: {}".format(positive_title)
#     print "negative titles: {}".format(negative_titles)
    
    target_features = title_to_feature_matrix(target_title)
    positive_features = title_to_feature_matrix(positive_title)
    n_negative_features = [title_to_feature_matrix(negative_title) for negative_title in negative_titles]
    
#     print "Target features shape: {}".format(target_features.shape)
#     print "Positive features shape: {}".format(positive_features.shape)
#     print "Negative features[0] shape: {}".format(n_negative_features[0].shape)
#     print "Num negative features: {}".format(len(n_negative_features)) 
    all_features = [target_features, positive_features] + n_negative_features
    features.append(all_features)


print "Done Loop"
features = np.array(features) 
targets = torch.LongTensor(len(features), 1).zero_()
training_dataset = data_utils.TensorDataset(torch.FloatTensor(features), targets)
train_loader = data_utils.DataLoader(training_dataset, batch_size = BATCH_SIZE, shuffle = True)
print "Succesfully made the data batcher"

0
Done Loop
Succesfully made the data batcher


In [ ]:
#-------------------------------------NET GLOBAL VARIABLES-------------------------------------#
MAX_TITLE_LENGTH, MAX_BODY_LENGTH = find_maximum_title_and_body_length(lookup)
KERNEL_SIZE = 3 #MAKE SURE THIS NUMBER IS ODD SO THAT THE PADDING MAKES SENSE
PADDING = (KERNEL_SIZE - 1) / 2
INPUT_SIZE = 200
HIDDEN_SIZE = 100
LEARNING_RATE = 1e-1
MARGIN = 1.0
NUM_EPOCHS = 25
BATCH_SIZE = 50
NUM_BATCHES = NUM_TRAINING_EXAMPLES/BATCH_SIZE

In [256]:
#-------------------------------------CNN-------------------------------------#
class CNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(200, hidden_dim, KERNEL_SIZE, padding = PADDING),
#             nn.ReLU()
            nn.Tanh()
        )
    def forward(self, x):
        x = self.layer1(x)
        return x

In [ ]:
INPUT_DIM = (MAX_TITLE_LENGTH, NUM_FEATURES_PER_WORD)

net = CNN(INPUT_SIZE, HIDDEN_SIZE)
criterion = nn.MultiMarginLoss(p=1, margin=MARGIN, weight = None, size_average=True) #HAHA just put these in to look smart 
optimizer = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)

cos = nn.CosineSimilarity(dim=1, eps=1e-6)

print "Beginning training..."
# ----TRAINING
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for idx,(sample, label) in enumerate(train_loader):
#     for idx in range(NUM_BATCHES):
        #Sample shape: [50, 22, 200, 38]
        #50 - Batch size, 22 - Num questions per data point
        sample = Variable(sample, requires_grad = True)
        label = Variable(label, requires_grad = True)

        #RE-ORDER DIMENSIONS OF THE SAMPLE
        sample = sample.permute(1, 0, 2, 3)

        batch_cos_similarities = []
        batch_num = idx
        
        target_question_features = sample[0] # 50 x 200 x 38
        positive_question_features = sample[1] # 50 x 200 x 38
        N_negative_question_features = sample[2:] #20 x 50 x 200 x 38
        
        #Determine lengths to know how many vectors to take the average across.
        target_question_lengths = find_start_of_padding_for_batch(target_question_features.data)
        positive_question_lengths = find_start_of_padding_for_batch(positive_question_features.data)
        N_negative_questions_lengths = [find_start_of_padding_for_batch(negative.data) for negative in N_negative_question_features]
        
        #RUN THROUGH NET
        target_question_net_output = net(target_question_features)
        positive_question_net_output = net(positive_question_features)
        N_negative_question_net_outputs = [net(negative) for negative in N_negative_question_features]
        
        #CREATE MASKS
        target_questions_masks = [create_mask(_) for _ in target_question_lengths] #DIM = 50 x 100 x 38
        positive_questions_masks = [create_mask(_) for _ in positive_question_lengths] #DIM = 50 x 100 x 38
        N_negative_questions_masks = [[create_mask(length) for length in length_list] #DIM = 50 x 20 x 100 x 38
                                      for length_list in N_negative_questions_lengths]
        
        #APPLY MASKS
        #Should the multiplicands, the masks, be Float Tensors or Variables? May have to be float tensors to ensure
        #    pytorch's directed graph back-prop is maintained.

        target_question_net_output_masked = target_question_net_output * Variable(torch.FloatTensor(target_questions_masks))
        positive_question_net_output_masked = positive_question_net_output * Variable(torch.FloatTensor(positive_questions_masks))
        N_negative_questions_net_output_masked = [N_negative_question_net_outputs[idx] * 
                                                  Variable(torch.FloatTensor(N_negative_questions_masks[idx]))
                                                  for idx in range(NUM_NEGATIVE_SAMPLES)]

        #SUM OVER WORDS
        target_question_net_output_masked_summed = torch.sum(target_question_net_output_masked, dim = 2) #DIM = 50 x 100
        positive_question_net_output_masked_summed = torch.sum(positive_question_net_output_masked, dim = 2) #DIM = 50 x 100
        N_negative_questions_net_output_masked_summed = [torch.sum(
                                                        N_negative_questions_net_output_masked[idx], dim = 2
                                                        )for idx in range(NUM_NEGATIVE_SAMPLES)] #DIM = 20 x 50 x 100

        
        cosine_similarity_pos = cos(target_question_net_output_masked_summed, positive_question_net_output_masked_summed)
        # ^ DIM = 50
        cosine_similarities_neg = [cos(target_question_net_output_masked_summed, N_negative_questions_net_output_masked_summed[idx])
                                  for idx in range(NUM_NEGATIVE_SAMPLES)]
        # ^ DIM = 20 x 50
        #

        cosine_similarities = torch.stack([cosine_similarity_pos] + cosine_similarities_neg) # DIM = 21 x 50
        
        optimizer.zero_grad()
        
        cosine_similarities = torch.t(cosine_similarities)
        label = torch.squeeze(label)
        loss = criterion(cosine_similarities, label)
        
        loss.backward()
        
        running_loss += loss.data[0]
        
        optimizer.step()

        if batch_num % 10 == 0:
            print "Epoch: {}, Batch: {}, Loss: {}".format(epoch, batch_num, loss.data[0])

    print "Loss after epoch " + str(epoch) + " :" + str(running_loss)
            


Beginning training...
Epoch: 0, Batch: 0, Loss: 0.893543541431
Epoch: 0, Batch: 10, Loss: 0.534539401531
Loss after epoch 0 :11.7748465836
Epoch: 1, Batch: 0, Loss: 0.456433653831
Epoch: 1, Batch: 10, Loss: 0.426854908466
Loss after epoch 1 :7.76640227437
Epoch: 2, Batch: 0, Loss: 0.372095197439
Epoch: 2, Batch: 10, Loss: 0.335641860962
Loss after epoch 2 :6.71314498782
Epoch: 3, Batch: 0, Loss: 0.320254087448
Epoch: 3, Batch: 10, Loss: 0.30139914155
Loss after epoch 3 :6.11533433199
Epoch: 4, Batch: 0, Loss: 0.289986848831
Epoch: 4, Batch: 10, Loss: 0.256373941898
Loss after epoch 4 :5.73690140247
Epoch: 5, Batch: 0, Loss: 0.223724097013
Epoch: 5, Batch: 10, Loss: 0.321817666292
Loss after epoch 5 :5.49702565372
Epoch: 6, Batch: 0, Loss: 0.229813978076
Epoch: 6, Batch: 10, Loss: 0.245957091451
Loss after epoch 6 :5.29262292385
Epoch: 7, Batch: 0, Loss: 0.254725188017
Epoch: 7, Batch: 10, Loss: 0.247336104512
Loss after epoch 7 :5.08241315186
Epoch: 8, Batch: 0, Loss: 0.23709821701
